In [40]:
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv")

#print(df.head(10))
print(df.shape)
print(df.info())

(891, 12)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB
None


## 🔱주요 컬럼:

PassengerId: 승객 ID

Survived: 생존 여부 (0 = 사망, 1 = 생존)

Pclass: 객실 등급 (1, 2, 3)

Name: 이름 (예: Braund, Mr. Owen Harris)

Sex: 성별 (male, female)

Age: 나이 (결측값 다수 포함)

SibSp: 함께 탑승한 형제자매/배우자 수

Parch: 함께 탑승한 부모/자녀 수

Ticket: 티켓 번호

Fare: 운임

Cabin: 선실 번호 (예: C85, 결측값 매우 많음)

Embarked: 탑승 항구 (S, C, Q)

💻 문제 1. (결측값 채우기 - 고급) Age 컬럼의 결측값(NaN)이 많습니다. 이 결측값을 **Pclass (객실 등급) 별 Age의 중앙값(median)**으로 채워 넣으려고 합니다. (예: 1등석(Pclass=1) 승객의 Age 결측값은 1등석 승객 전체의 Age 중앙값으로 채움) 결측값을 모두 채운 후, Age 컬럼의 전체 평균값을 구하시오. (단, 소수점 셋째 자리에서 반올림하여 둘째 자리까지 계산)

In [41]:
# 1번 문제(transform)
# df.groupby('<group_col>')['<calc_col>'].transform('<function>')

age_fill_values = df.groupby('Pclass')['Age'].transform('median')
df['Age'] = df['Age'].fillna(age_fill_values)

result = df['Age'].mean()
print(round(result, 2))
print(age_fill_values)


29.07
0      24.0
1      37.0
2      24.0
3      37.0
4      24.0
       ... 
886    29.0
887    37.0
888    24.0
889    37.0
890    24.0
Name: Age, Length: 891, dtype: float64


In [42]:
# 1번 문제(loc)
# df.loc[<row_condition>, <column_name>] = new_value

medians = df.groupby('Pclass')['Age'].median()
# medians[1] = 1등석 중앙값
# medians[2] = 2등석 중앙값
# medians[3] = 3등석 중앙값

df.loc[(df['Pclass'] == 1) & (df['Age'].isna()), 'Age'] = medians[1]
df.loc[(df['Pclass'] == 2) & (df['Age'].isna()), 'Age'] = medians[2]
df.loc[(df['Pclass'] == 3) & (df['Age'].isna()), 'Age'] = medians[3]

result = df['Age'].mean()
print(round(result, 2))

print(medians[1])

29.07
37.0


🚀 요약: 그래서 언제 뭘 쓸까?
loc (수술): "딱 이 칸, 저 칸만 골라서" 값을 바꾸고 싶을 때.

df.loc[<조건>, <바꿀컬럼>] = <새값>

transform (공정): "그룹별 계산 값을" "원본 모든 행에" 매핑해서 사용하고 싶을 때.

df.groupby(...).transform(...) (주로 fillna와 찰떡궁합입니다.)

💻 문제 2. (문자열 추출 - Regex) Name 컬럼에는 승객의 호칭(Title)이 포함되어 있습니다. (예: Mr., Mrs., Miss., Master. 등) Name 컬럼에서 **정규표현식(Regex)**을 사용하여 이 호칭([A-Za-z]+\.)을 추출하여 'Title'이라는 새 컬럼을 만드시오. 그 후, 'Title'별 인원수를 집계할 때, 10명 이상의 인원을 가진 'Title' 그룹들의 Age 평균을 구하시오. (단, 소수점 셋째 자리에서 반올림하여 둘째 자리까지 계산)

In [43]:
# 2번 문제
df['Title'] = df['Name'].str.extract('([A-Za-z]+)\.')
title_counts = df['Title'].value_counts()

target_titles = title_counts[title_counts >= 10].index
target_df = df[df['Title'].isin(target_titles)]
result = target_df['Age'].mean()
print(round(result, 2))

28.66


💻 문제 3. (결측값 + 문자열 처리 - 복합) Cabin 컬럼은 결측값이 매우 많습니다.

Cabin 컬럼의 결측값은 'Unknown'으로 채웁니다.

Cabin 컬럼이 결측값이 아닌 경우, 값의 **첫 번째 알파벳(Deck)**만 추출합니다. (예: C85 -> C, G6 -> G)

Unknown으로 채워진 데이터는 Deck를 'U'로 변경합니다. (힌트: fillna('U') 후 .str[0])

이렇게 생성된 Deck 컬럼(A, B, C... U)별 Survived (생존율) 평균을 계산할 때, 생존율이 가장 높았던 Deck의 이름을 출력하시오.

In [44]:
# 3번 문제
df['Cabin'] = df['Cabin'].fillna('Unknown')
#print(df['Cabin'].head(10))

q1 = df[['Cabin', 'Survived']]

deck = q1['Cabin'].str[0]
q1['Cabin'] = deck

survived_rates = q1.groupby('Cabin')['Survived'].mean()
print(survived_rates)


Cabin
A    0.466667
B    0.744681
C    0.593220
D    0.757576
E    0.750000
F    0.615385
G    0.500000
T    0.000000
U    0.299854
Name: Survived, dtype: float64


C:\Users\sangh\AppData\Local\Temp\ipykernel_21652\403286857.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  q1['Cabin'] = deck


In [45]:
# 3번 문제 완성 코드
df['Deck'] = df['Cabin'].fillna('U').str[0]

deck_survival_rates = df.groupby('Deck')['Survived'].mean()

print(deck_survival_rates)

Deck
A    0.466667
B    0.744681
C    0.593220
D    0.757576
E    0.750000
F    0.615385
G    0.500000
T    0.000000
U    0.299854
Name: Survived, dtype: float64


🚀 핵심: Survived 컬럼 (0과 1로만 구성)의 .mean()을 구하면, 그게 바로 1의 비율, 즉 생존율이 된다. (예: [1, 0, 1]의 평균은 0.66 -> 66%)

💻 문제 4. (Feature 생성 및 Categorization) SibSp (형제/배우자)와 Parch (부모/자녀) 컬럼을 합쳐, 본인을 포함한 'FamilySize' (가족 총원 = SibSp + Parch + 1) 컬럼을 생성합니다. 이 'FamilySize'를 기준으로 다음과 같이 'FamilyGroup'을 분류합니다.

1명: 'Alone'

2~4명: 'Small'

5명 이상: 'Large' 

'FamilyGroup'별 Survived (생존율) 평균을 계산하고, 생존율이 가장 높았던 그룹의 이름을 출력하시오.

In [46]:
# 4번 문제
df = pd.read_csv("https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv")

df['FamilySize'] = df['SibSp'] + df['Parch'] + 1

df['FamilyGroup'] = 'Unknown'

df.loc[df['FamilySize']==1, 'FamilyGroup'] = 'Alone'
df.loc[(df['FamilySize'] >= 2) & (df['FamilySize'] <= 4), 'FamilyGroup'] = 'Small'
df.loc[df['FamilySize']>=5, 'FamilyGroup'] = 'Large'

result = df.groupby('FamilyGroup')['Survived'].mean()
print(result)





FamilyGroup
Alone    0.303538
Large    0.161290
Small    0.578767
Name: Survived, dtype: float64


💻 문제 5. (복합 조건 apply 및 Feature 생성) Age 컬럼 (결측값이 10번 문제 방식으로 채워졌다고 가정)과 Sex 컬럼을 조합하여 'AgeSexGroup'을 만듭니다.

'male'이면서 18세 미만: 'Boy'

'female'이면서 18세 미만: 'Girl'

'male'이면서 18세 이상: 'Man'

'female'이면서 18세 이상: 'Woman' 

가장 많은 인원수를 차지하는 'AgeSexGroup'의 이름을 출력하시오.

In [47]:
# 결측값 처리
age_fill_values = df.groupby('Pclass')['Age'].transform('median')
df['Age'] = df['Age'].fillna(age_fill_values)

# 5번 문제
df['AgeSexGroup'] = 'Unknown'

df.loc[(df['Sex']=='male') & (df['Age'] < 18), 'AgeSexGroup'] = 'Boy'
df.loc[(df['Sex']=='female') & (df['Age'] < 18), 'AgeSexGroup'] = 'Girl'
df.loc[(df['Sex']=='male') & (df['Age'] >= 18), 'AgeSexGroup'] = 'Man'
df.loc[(df['Sex']=='female') & (df['Age'] >= 18), 'AgeSexGroup'] = 'Woman'

result = df['AgeSexGroup'].value_counts()
print(result)




AgeSexGroup
Man      519
Woman    259
Boy       58
Girl      55
Name: count, dtype: int64


💻 문제 6. (Pivot Table 및 .loc) (10번 문제의 Age 결측값 처리, 12번 문제의 Deck('U' 포함) 처리가 완료되었다고 가정합니다.) pivot_table을 사용하여 'Deck'을 index로, 'Pclass'를 columns로, Age의 평균을 values로 하는 피벗 테이블을 생성합니다. 생성된 피벗 테이블에서, 'U' (Unknown) 데크의 3등석(Pclass=3) 승객 평균 나이를 소수점 셋째 자리에서 반올림하여 둘째 자리까지 출력하시오.

In [48]:
# 결측값 처리
df['Deck'] = df['Cabin'].fillna('U').str[0]

# 6번 문제
pivot_df = df.pivot_table(index='Deck', columns='Pclass', values='Age', aggfunc='mean')

print(round(pivot_df.loc['U', 3], 2))

24.91


### 💻 제2유형: 데이터 모형 구축
타이타닉 생존자 예측
[훈련 데이터(train.csv)로 학습한 모델을 테스트 데이터(test.csv)에 적용하여 예측한 결과를 제출하시오.

Target: Survived (생존 여부, 이진 분류, {0, 1})

제출형식: result.csv

ID, pred 칼럼만 존재해야 합니다.

(주의: test.csv의 PassengerId를 ID 칼럼으로 사용해야 함)

평가지표: Accuracy (정확도) ]

In [ ]:
train_url = "https://raw.githubusercontent.com/agconti/kaggle-titanic/master/data/train.csv"
test_url = "https://raw.githubusercontent.com/agconti/kaggle-titanic/master/data/test.csv"

train = pd.read_csv(train_url)
test = pd.read_csv(test_url)

# 결측치 처리
#print(test.info())
# trian: age, cabin, embarked
# test: age, cabin

age_fill_values_train = train.groupby('Pclass')['Age'].transform('median')
train['Age'] = train['Age'].fillna(age_fill_values_train)

age_fill_values_test = test.groupby('Pclass')['Age'].transform('median')
test['Age'] = test['Age'].fillna(age_fill_values_test)

train['Cabin'] = train['Cabin'].fillna('Unknown')
test['Cabin'] = test['Cabin'].fillna('Unknown')

fill_embarked = train['Embarked'].mode()[0]
train['Embarked'] = train['Embarked'].fillna(fill_embarked)

test['Fare'] = test['Fare'].fillna(test['Fare'].mean())

# 인코딩
#print(test.info())
# train 인코딩: Sex, Cabin
# train 제거: PassengerId, Ticket, Embarked

X_train = train.drop(['Survived', 'PassengerId', 'Ticket', 'Embarked'], axis=1)
y_train = train['Survived']

X_test = test.drop(['PassengerId', 'Ticket', 'Embarked'], axis=1)

cols = ['Sex', 'Cabin']
X_train = pd.get_dummies(X_train, columns=cols)
X_test = pd.get_dummies(X_test, columns=cols)



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass       418 non-null    int64  
 2   Name         418 non-null    object 
 3   Sex          418 non-null    object 
 4   Age          418 non-null    float64
 5   SibSp        418 non-null    int64  
 6   Parch        418 non-null    int64  
 7   Ticket       418 non-null    object 
 8   Fare         418 non-null    float64
 9   Cabin        418 non-null    object 
 10  Embarked     418 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 36.1+ KB
None
